# TRAINING

In [6]:
import sagemaker

sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

In [7]:
from sagemaker.tensorflow import TensorFlow

hyperparams={'epochs'              : 50,
             'layers'              : 'heads', # heads or all
             'pretrained_weight'   : 'imagenet', # imagenet, coco
            }

bucket_name = sagemaker_session.default_bucket()
output_path = f's3://{bucket_name}/jobs'

In [8]:
import os

os.listdir("train/")

['.ipynb_checkpoints',
 'annotations.json',
 'requirements.txt',
 'model.py',
 'utils.py',
 'model_serving.py',
 'mrcnn',
 'preprocessor.py',
 'train.py']

In [9]:
tf_estimator = TensorFlow(entry_point          = 'train.py', 
                          source_dir           = 'train/',
                          output_path          = f'{output_path}/',
                          code_location        = output_path,
                          role                 = role,
                          instance_count       = 1, 
                          instance_type        = 'ml.p3.2xlarge',
                          framework_version    = '1.15.3', 
                          py_version           = 'py3',
                          script_mode          = True,
                          sagemaker_session    = sagemaker_session,
                          hyperparameters      = hyperparams)

In [11]:
# import time
# job_name=f'tensorflow-gpu-car-damage-detection-{time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())}'
# tf_estimator.fit({'training'  : "s3://sagemaker-us-east-2-186402456289/sagemaker/mask_rcnn_localization_train",
#                   'validation': "s3://sagemaker-us-east-2-186402456289/sagemaker/mask_rcnn_localization_valid",
#                   }, 
#                  job_name = job_name)

# DEPLOYMENT

In [31]:
import sagemaker
from sagemaker.tensorflow.model import TensorFlowModel

sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

In [32]:
# s3://sagemaker-us-east-2-186402456289/jobs/tensorflow-gpu-car-damage-detection-2021-09-09-13-51-00/output/model.tar.gz

artifact_dir = "s3://sagemaker-us-east-2-186402456289/jobs/tensorflow-gpu-car-damage-detection-2021-09-27-11-27-50/output/model.tar.gz"

In [33]:
estimator = TensorFlowModel(model_data= artifact_dir,
                            role=role, 
                            framework_version="1.15.3",
                            entry_point = 'inference.py',
                            source_dir="code",
                            env = {'PREDICT_USING_GRPC' : 'false'})

In [34]:
predictor = estimator.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

update_endpoint is a no-op in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


-------!

In [35]:
predictor.endpoint_name

'tensorflow-inference-2021-10-03-20-43-56-962'

# PREDICTION

In [36]:
import visualizer as visualize
import skimage

In [37]:
import os
import numpy as np
list_files = os.listdir("../sample_files") 

In [27]:
list_files[0:1]

['car wreck_-ef43703c49fb57a010fa9ef1c6c28dbeffd491793e7333bb4299fe4490d95a7f.jpg']

In [38]:
import boto3
import json
import numpy as np
from PIL import Image
from IPython.display import display
import io
import base64
from io import BytesIO

categories = np.array([0,'none','low', 'medium', 'high'])

def client_predictor(files):
    for file in files:
        image_path = f"../sample_files/{file}"
        car_image = open(image_path, 'rb').read()
        endpoint_name = predictor.endpoint_name
        runtime_client = boto3.client('runtime.sagemaker')
        response = runtime_client.invoke_endpoint(EndpointName=endpoint_name, 
                                           ContentType='application/x-image',     
                                           Body=car_image)
        res = response['Body'].read().decode('ascii')
        
        r = json.loads(res)
        print(r)
#         viz_img = r['image']
#         decoded = base64.b64decode(viz_img)
#         image = io.BytesIO(decoded)
# #         image_64_decode = base64.decodebytes(image) 

#         img = Image.open(image)
#         display(img)

# VISUALIZE

In [39]:
client_predictor(list_files[0:1])

ModelError: An error occurred (ModelError) when calling the InvokeEndpoint operation: Received server error (502) from primary with message "<html>
<head><title>502 Bad Gateway</title></head>
<body>
<center><h1>502 Bad Gateway</h1></center>
<hr><center>nginx/1.18.0</center>
</body>
</html>
". See https://us-east-2.console.aws.amazon.com/cloudwatch/home?region=us-east-2#logEventViewer:group=/aws/sagemaker/Endpoints/tensorflow-inference-2021-10-03-20-43-56-962 in account 186402456289 for more information.

In [66]:
# detect(predictor,list_files)

# HOUSE CLEANING

In [30]:
predictor.delete_endpoint()